Stratified 5 fold splitting based on nodetype as y so that we will have same node type distribution as original dataset in train and test set

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

from collections import Counter
import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data_path = "/content/gdrive/My Drive/Loopy_belief_propagation/Graph_neural_network/Data/"
df_user = pd.read_csv(data_path+"politifact_shu_user_data_formated.csv")
df_news = pd.read_csv(data_path+"politifact_shu_news_data_formated.csv")
df_source = pd.read_csv(data_path+"politifact_shu_source_data_formated.csv")
df_followers_sample = pd.read_csv(data_path+"politifact_shu_follower_data_formated_with_features_only.csv")
df_all_user_news_pair = pd.read_csv(data_path+"politifact_shu_user_news_pair_data_formated.csv")
df_all_source_news_pair = pd.read_csv(data_path+"politifact_shu_source_news_pair_data_formated.csv")

In [ ]:
df_user.shape, df_news.shape, df_source.shape, df_all_user_news_pair.shape, df_all_source_news_pair.shape,df_followers_sample.shape

((1135, 246), (936, 149), (88, 5), (401715, 3), (1081, 2), (4118, 2))



# select news from pairs that have features

In [ ]:
df_all_source_news_pair.source_id.nunique(), df_all_source_news_pair.news_id.nunique()

(292, 990)

In [ ]:
df_all_user_news_pair.user_id.nunique(), df_all_user_news_pair.news_id.nunique()

(295469, 701)

In [ ]:
df_all_user_news_pair = df_all_user_news_pair.astype(str)
df_user.user_id = df_user.user_id.astype(str)
df_user_news_pair = df_all_user_news_pair[df_all_user_news_pair.user_id.isin(df_user.user_id)]


In [ ]:
df_user_news_pair.shape, df_user_news_pair.user_id.nunique(), df_user_news_pair.news_id.nunique()

((15511, 3), 1135, 544)

In [ ]:
df_news = df_news[df_news.news_id.isin(df_user_news_pair.news_id)]
df_news.shape, df_news.news_id.nunique()

((516, 149), 516)

select only those pairs where news are shared by user and have features

In [ ]:
df_user_news_pair = df_user_news_pair[df_user_news_pair.news_id.isin(df_news.news_id)]

In [ ]:
df_user_news_pair.shape, df_user_news_pair.user_id.nunique(), df_user_news_pair.news_id.nunique()

((14320, 3), 1135, 516)

In [ ]:
df_source_news_pair = df_all_source_news_pair[df_all_source_news_pair.news_id.isin(df_news.news_id)]
df_source_news_pair.shape, df_source_news_pair.news_id.nunique()

((571, 2), 516)

# Add source bias and news

In [ ]:
df_news_source_added = df_news.merge(df_source_news_pair, on='news_id', how='left')
df_news_source_added = df_news_source_added.merge(df_source[['source_id','bias']], on='source_id', how='left')
df_news_source_added.shape

(571, 151)

In [ ]:
df_news_source_added.columns

Index(['news_id', 'news_title', 'news_text', 'source', 'news_publish_date',
       'news_top_img', 'news_author', 'news_label', 'WC', 'Analytic',
       ...
       'Sadness', 'Surprise', 'Trust', 'Objective', 'compound', 'neg', 'neu',
       'pos', 'source_id', 'bias'],
      dtype='object', length=151)

In [ ]:
df_news_source_added[df_news_source_added.duplicated(subset=['news_id','bias'])]
df_news_source_added.drop_duplicates(subset=['news_id','source_id'], inplace=True)

In [ ]:
df_news_source_added.shape, df_news_source_added.news_id.nunique(), df_news_source_added.source_id.nunique()

((516, 151), 516, 198)

# select only those sources who published 516 news

In [ ]:
df_source = df_source[df_source.source_id.isin(df_news_source_added.source_id)]

In [ ]:
df_source.shape, df_source.source_id.nunique()

((71, 5), 71)

In [ ]:
df_user.user_id = df_user.user_id.astype(str)
df_news.news_id = df_news.news_id.astype(str)
df_source.source_id = df_source.source_id.astype(str)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
df_user.drop_duplicates(subset=['user_id'], inplace=True)
df_news.drop_duplicates(subset=['news_id'], inplace=True)
df_source.drop_duplicates(subset=['source_id'], inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
all_enitities = np.concatenate((df_user.user_id.unique(),df_news.news_id.unique(),df_source.source_id.unique()))

In [ ]:
df_AllEntities = pd.DataFrame(all_enitities, columns=['ids'])

In [ ]:
df_AllEntities

,ids
0,759251
1,817652
2,1653051
3,2097571
4,2836421
...,...
1717,source254
1718,source258
1719,source272
1720,source281


In [ ]:
df_AllEntities['entity_label'] = df_AllEntities.ids.apply(lambda x: 'news' if x.startswith('politifact') else ('source' if x.startswith('source') else 'user'))

In [ ]:
df_AllEntities

,ids,entity_label
0,759251,user
1,817652,user
2,1653051,user
3,2097571,user
4,2836421,user
...,...,...
1717,source254,source
1718,source258,source
1719,source272,source
1720,source281,source


In [ ]:
df_AllEntities.groupby('entity_label').count()

,ids
entity_label,
news,516
source,71
user,1135


In [ ]:
y = df_AllEntities.entity_label.values

In [ ]:
df_AllEntities.shape

(1722, 2)

In [ ]:
sk = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
dest_path = "/content/gdrive/My Drive/Loopy_belief_propagation/Graph_neural_network/code/code_for_516_news/5_fold/"
if not os.path.exists(dest_path+'/train_test'):
      os.makedirs(dest_path+'/train_test')
for i,(train_index, test_index) in enumerate(sk.split(np.zeros(len(y)), y)):
    train = df_AllEntities.loc[train_index]
    test = df_AllEntities.loc[test_index]

    print(train.shape, test.shape)
    print(Counter(train.entity_label), Counter(test.entity_label))
    if not os.path.exists(dest_path+'/train_test/fold'+str(i)):
      os.makedirs(dest_path+'/train_test/fold'+str(i))
    train.to_csv(dest_path+'/train_test/fold'+str(i)+"/train.csv", index=False)
    test.to_csv(dest_path+'/train_test/fold'+str(i)+"/test.csv", index=False)

(1377, 2) (345, 2)
Counter({'user': 908, 'news': 412, 'source': 57}) Counter({'user': 227, 'news': 104, 'source': 14})
(1377, 2) (345, 2)
Counter({'user': 908, 'news': 413, 'source': 56}) Counter({'user': 227, 'news': 103, 'source': 15})
(1378, 2) (344, 2)
Counter({'user': 908, 'news': 413, 'source': 57}) Counter({'user': 227, 'news': 103, 'source': 14})
(1378, 2) (344, 2)
Counter({'user': 908, 'news': 413, 'source': 57}) Counter({'user': 227, 'news': 103, 'source': 14})
(1378, 2) (344, 2)
Counter({'user': 908, 'news': 413, 'source': 57}) Counter({'user': 227, 'news': 103, 'source': 14})
